# SQL Options in PySpark
* Notebook by Adam Lang
* Date: 4/8/2025

# Overview
* PySpark provides 2 main options when it comes to using the SQL language:
  1. Spark SQL
  2. SQL Transformer

## 1. Spark SQL

* Spark TempView provides 2 functions that allow users to run **SQL** queries against a Spark DataFrame:

 - **createOrReplaceTempView:**
    * The lifetime of this temporary view is tied to the SparkSession that was used to create the dataset.
    * Creates (or replaces if that view name already exists) a lazily evaluated "view" that you can then use like a hive table in Spark SQL.
    * **It does not persist to memory unless you cache the dataset that underpins the view.**
 - **createGlobalTempView:**
    * The lifetime of this temporary view is tied to this Spark application.
    * This feature is useful when you want to share data among different sessions and keep alive until your application ends.

A **Spark Session vs. Spark application:**

**Spark application** can be used:

  1. for a single batch job
  2. interactive session with multiple jobs
  3. long-lived server continually satisfying requests
  4. Spark job can consist of more than just a single map and reduce.
  5. Consists of **more than one Spark Session.**

A **SparkSession** on the other hand:

  1. Interaction between two or more entities.
  2. Can be created without creating SparkConf, SparkContext or SQLContext, (they’re encapsulated within the SparkSession which is new to Spark 2.0)


## 2. SQL Transformer

* You can leverage the SQL transformer option to write your own free-form SQL scripts.

# SQL Options within regular PySpark calls

1. The expr function in PySparks SQL Function Library
2. PySparks selectExpr function

We will go over all these in detail so buckel up!



---
# 1. Spark SQL Implementation
* Now lets implement Spark SQL.

In [2]:
## init spark session
import pyspark
from pyspark.sql import SparkSession

# create spark object
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
spark

## Dataset
* This is a dataset of crime in England and Wales.

In [4]:
## set data path
path = '/content/drive/MyDrive/Colab Notebooks/PySpark Data Science/Spark_Dataframes/'

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
## load data
crime_data = spark.read.csv(path+"rec-crime-pfa.csv",
                            header=True,
                            inferSchema=True)

## view data
crime_data.limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Rolling year total number of offences
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561


In [7]:
## lets view schema
crime_data.printSchema()

root
 |-- 12 months ending: string (nullable = true)
 |-- PFA: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Offence: string (nullable = true)
 |-- Rolling year total number of offences: integer (nullable = true)



In [8]:
## create new df with renamed column
df = crime_data.withColumnRenamed('Rolling year total number of offences', 'Count')
df.printSchema()

root
 |-- 12 months ending: string (nullable = true)
 |-- PFA: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Offence: string (nullable = true)
 |-- Count: integer (nullable = true)



Summary
* Count is now the new name of the last column in the df.

## Create TempView for Basic SQL Queries

In [9]:
## create a temp view
df.createOrReplaceTempView('tempview')

In [11]:
## SQL query off tempview
spark.sql('SELECT * FROM tempview').limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Count
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561


In [12]:
## SQL query off tempview
spark.sql('SELECT * FROM tempview WHERE Count > 1000').limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Count
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561
4,31/03/2003,Avon and Somerset,South West,Drug offences,2308


In [13]:
## SQL query off tempview
spark.sql('SELECT Region, Offence FROM tempview WHERE Count> 1000').limit(5).toPandas()

,Region,Offence
0,South West,All other theft offences
1,South West,Bicycle theft
2,South West,Criminal damage and arson
3,South West,Domestic burglary
4,South West,Drug offences


In [14]:
## pass to an object
sql_results = spark.sql('SELECT Region, Offence FROM tempview WHERE Count > 1000')
sql_results.limit(5).toPandas()

,Region,Offence
0,South West,All other theft offences
1,South West,Bicycle theft
2,South West,Criminal damage and arson
3,South West,Domestic burglary
4,South West,Drug offences


In [16]:
## groupby
spark.sql('SELECT Region, sum(Count) AS Total FROM tempview GROUP BY Region ORDER BY Total DESC').toPandas()

,Region,Total
0,London,42691902
1,South East,30911995
2,North West,30235732
3,Yorkshire and The Humber,23769180
4,West Midlands,21040955
5,East,19890612
6,South West,17985880
7,East Midlands,17264827
8,Wales,11137260
9,North East,10236642


---
# 2. SQL Transformer Implementation

In [17]:
## read in imports
from pyspark.ml.feature import SQLTransformer

In [18]:
## create object with SQL call --> __THIS__ is a placeholder for what you are transforming
sqlTrans = SQLTransformer(statement="SELECT PFA, Region, Offence FROM __THIS__")

In [21]:
## now transform
sqlTrans.transform(df).show(5)

+-----------------+----------+--------------------+
|              PFA|    Region|             Offence|
+-----------------+----------+--------------------+
|Avon and Somerset|South West|All other theft o...|
|Avon and Somerset|South West|       Bicycle theft|
|Avon and Somerset|South West|Criminal damage a...|
|Avon and Somerset|South West|Death or serious ...|
|Avon and Somerset|South West|   Domestic burglary|
+-----------------+----------+--------------------+
only showing top 5 rows



In [22]:
## type of sqlTransformer
type(sqlTrans)

pyspark.ml.feature.SQLTransformer

In [26]:
## create object with SQL call --> __THIS__ is a placeholder for what you are transforming

sqlTrans = SQLTransformer(statement= "SELECT Offence, SUM(Count) as Total FROM __THIS__ GROUP BY Offence")

In [27]:
## transform
sqlTrans.transform(df).show()

+--------------------+--------+
|             Offence|   Total|
+--------------------+--------+
|Public order offe...|10925676|
|       Bicycle theft| 5297006|
|Residential burglary| 1671469|
|Violence without ...|16590158|
|All other theft o...|30979393|
|             Robbery| 3788128|
|               CIFAS| 7678981|
|      Fraud offences| 2596554|
|     Sexual offences| 4006741|
|Criminal damage a...|37767463|
|            Homicide|   34154|
|Possession of wea...| 1555951|
|          UK Finance| 2925861|
|Stalking and hara...| 5587434|
|Theft from the pe...| 5105153|
|         Shoplifting|16781641|
|       Drug offences| 9999435|
|    Vehicle offences|26075670|
|   Domestic burglary|11694636|
|Miscellaneous cri...| 3143136|
+--------------------+--------+
only showing top 20 rows



In [31]:
sqlTrans = SQLTransformer(statement= "SELECT SUM(Count) as Total FROM __THIS__")
sqlTrans.transform(df).show()

+---------+
|    Total|
+---------+
|244720928|
+---------+



In [29]:
## read into df
result_df = sqlTrans.transform(df)
result_df.show()

+--------------------+--------+
|             Offence|   Total|
+--------------------+--------+
|Public order offe...|10925676|
|       Bicycle theft| 5297006|
|Residential burglary| 1671469|
|Violence without ...|16590158|
|All other theft o...|30979393|
|             Robbery| 3788128|
|               CIFAS| 7678981|
|      Fraud offences| 2596554|
|     Sexual offences| 4006741|
|Criminal damage a...|37767463|
|            Homicide|   34154|
|Possession of wea...| 1555951|
|          UK Finance| 2925861|
|Stalking and hara...| 5587434|
|Theft from the pe...| 5105153|
|         Shoplifting|16781641|
|       Drug offences| 9999435|
|    Vehicle offences|26075670|
|   Domestic burglary|11694636|
|Miscellaneous cri...| 3143136|
+--------------------+--------+
only showing top 20 rows



In [30]:
# Parses the expression string into the column that it represents
from pyspark.sql.functions import expr

In [32]:
## add new column into dataframe = count/total
df.withColumn('percent', expr('round((count/244720928)*100, 2)')).show()

+----------------+-----------------+----------+--------------------+-----+-------+
|12 months ending|              PFA|    Region|             Offence|Count|percent|
+----------------+-----------------+----------+--------------------+-----+-------+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|25959|   0.01|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft| 3090|    0.0|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|26202|   0.01|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|    2|    0.0|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|14561|   0.01|
|      31/03/2003|Avon and Somerset|South West|       Drug offences| 2308|    0.0|
|      31/03/2003|Avon and Somerset|South West|      Fraud offences| 5339|    0.0|
|      31/03/2003|Avon and Somerset|South West|            Homicide|   19|    0.0|
|      31/03/2003|Avon and Somerset|South West|Miscellaneous cri...| 1597|    0.0|
|   

In [36]:
## select all cols in df instead
df.select("*", expr('round((count/244720928)*100, 2)')).toPandas().head(5)

,12 months ending,PFA,Region,Offence,Count,"round(((count / 244720928) * 100), 2)"
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959,0.01
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090,0.00
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202,0.01
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2,0.00
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561,0.01


In [41]:
## selectEXPR
df.selectExpr("*", 'round((count/244720928)*100, 2) AS Percent').filter("Region = 'South West'").toPandas().head(5)

,12 months ending,PFA,Region,Offence,Count,Percent
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959,0.01
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090,0.00
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202,0.01
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2,0.00
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561,0.01
